In [55]:
import pandas as pd

In [56]:
labeled_misc = pd.read_csv("Manual_labeling_file_filled.csv")

In [57]:
labeled_misc

,Reviews,Labeler,Useful?,Comments
0,I give it 5 stars it almost had everything I ...,Romario,1,NaN
1,I like this app and I segues to make stronger ...,Romario,1,NaN
2,So far does everything I want it to. \n,Romario,0,NaN
3,This app provides a lot of value for my phone ...,Romario,0,NaN
4,This is a wonderful app for figuring out what ...,Romario,0,NaN
...,...,...,...,...
995,Thanks Great app does he simple things well. ...,Joanna,1,NaN
996,You said you fixed the tracing of comet ISON I...,Joanna,1,NaN
997,This App is by far my Go to App for doing art....,Joanna,1,NaN
998,Happy to see how fast the crashing issue was f...,Joanna,0,NaN


# Clean up ones we did not label

In [58]:
labeled_misc = labeled_misc[labeled_misc["Useful?"].isin(["0","1"])]

In [59]:
labeled_misc["Useful?"] = labeled_misc["Useful?"].astype(int)

C:\Users\Romario\AppData\Local\Temp\ipykernel_16984\2301899026.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeled_misc["Useful?"] = labeled_misc["Useful?"].astype(int)


In [60]:
labeled_misc["Useful?"].value_counts()

0    866
1    126
Name: Useful?, dtype: int64

In [61]:
additional = labeled_misc[labeled_misc["Useful?"]==0].shape[0] - labeled_misc[labeled_misc["Useful?"]==1].shape[0]
print("Additional Required positive examples to balance dataset: ", additional)

Additional Required positive examples to balance dataset:  740


In [62]:
labeled_misc["Original_type"] = "mis"

C:\Users\Romario\AppData\Local\Temp\ipykernel_16984\4091443445.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeled_misc["Original_type"] = "mis"


# Balance with useful reviews from the additional dataset:

In [63]:
emse_df = pd.read_csv("emse19/emse19/emse19/Training Dataset/review_and_label.csv")

In [64]:
emse_df["misc"] = emse_df["Label"].apply(lambda x: "mis" in x.lower())

In [65]:
emse_df = emse_df[~emse_df["misc"]]

In [66]:
emse_df["Useful?"] = 1

In [67]:
emse_df = emse_df.sample(additional, random_state=0)

In [68]:
emse_df.rename(columns={"Label":"Original_type"}, inplace=True)

In [69]:
emse_df["Original_type"] = emse_df["Original_type"].str.strip()

In [70]:
emse_df = emse_df[["Reviews", "Original_type", "Useful?"]]

In [71]:
final_dataset = pd.concat([labeled_misc, emse_df],  ignore_index=True)

In [72]:
final_dataset["Useful?"].value_counts()

1    866
0    866
Name: Useful?, dtype: int64

In [73]:
final_dataset.to_csv("balanced_augmentation_dataset.csv", index=None)